In [1]:
import numpy as np
import keras
from keras.models import Model
from model import ResNet, pyramid_pooling_module, deconvolution_module
from keras.layers import Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import os,sys
from ImageDataGenerator import ImageGenerator
from sklearn.model_selection import train_test_split


def train_val_generator(train_dir, label_dir, batch_size):
    
    
    X_train,X_val,y_train,y_val = train_test_split(os.listdir(train_dir),
                                                   os.listdir(label_dir),
                                                   shuffle=False,
                                                   test_size=0.2)
    assert len(X_train)==len(y_train),'Number of images is not equal to number of labels'
    
    train_generator = ImageGenerator(image_list=X_train,
                                     label_list=y_train,
                                     image_dir=train_dir,
                                     anno_dir=label_dir,
                                     batch_size=batch_size)
    
    val_generator = ImageGenerator(image_list=X_valid,
                                   label_dir=y_valid,
                                   image_dir=train_dir,
                                   anno_dir=label_dir,
                                   batch_size=batch_size)
    
    return (train_generator,val_generator)


def build_PSPNet(input_shape, num_channels, out_shape,
                 num_classes, learn_rate, loss_function='mean_squared_error'):
    print('Started building model\n')
    input_layer = Input((input_shape[0],input_shape[1],num_channels))
    resnet_block = ResNet(input_layer)
    spp_block = pyramid_pooling_module(resnet_block)
    out_layer = deconvolution_module(concat_layer=spp_block,
                                    num_classes=num_classes,
                                    output_shape=out_shape)
    
    model = Model(inputs=input_layer,outputs=out_layer)
    
    adam = Adam(learning_rate=learn_rate)
    
    model.compile(optimizer=adam,
                  loss=loss_function,
                  metrics=['accuracy'])
    print('Model has compiled\n')
    print('The input shape will be {} and the output of'
          ' the model will be {}'.format(model.input_shape[1:],model.output_shape[1:]))
    return model


def train_model(model, filepath, train_generator, val_generator,
                epochs=50000, steps_per_epoch=250):
    
    checkpoint = ModelCheckpoint(filepath, save_best_only=False, verbose=1)
    
    print('Model will be saved in' 
          ' directory: {} as {}\n'.format(os.path.split(filepath)[0],os.path.split(filepath)[1]))
    
    model.fit_generator(train_generator,
                        validation_data=val_generator,
                        callbacks=[checkpoint],
                        epochs=epochs,verbose=1,
                        steps_per_epoch=steps_per_epoch)
    
    print('Finished training model. Exiting function ...\n')
    
    return model

Using TensorFlow backend.
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/viha4393/anaconda36/lib/python3.7/site-pa